# HydroHomies Plots
In this notebook, the plots, figures and also some explanations or details about each of them are being presented.  

To clarify plots, please follow this order:
- Title for each plot is mandatory
- Analysis must be written 
- legends are manedatory

### Importing the needed modules

In [25]:
import yaml
import pandas as pd
import numpy as np

from bokeh.plotting import figure, show
from bokeh.io import show, output_notebook
from bokeh.transform import dodge, factor_cmap
from bokeh.models import ColumnDataSource, FactorRange, Whisker
import panel as pn
from scipy.stats import sem
from bokeh.transform import factor_cmap
from bokeh.layouts import gridplot


output_notebook()
pn.extension()

import hvplot.pandas

Loading BokehJS ...

### Loading all data


In [26]:
with open('config.yaml') as stream:
    config = yaml.safe_load(stream)

### Cleaning (Digit Span Raw Data)

In [27]:
def clean_digit_span(raw_df):
    # Select the sequence length data from the raw data and create a dataframe
    seq_length_df = raw_df[raw_df[1].astype(str).str.match(r'\d+')]

    # Get the value of the longest sequence remembered
    longest = seq_length_df[2]
    longest = longest.tolist()

    # Get the number of errors made
    error_number = seq_length_df[3]
    error_number = error_number.tolist()

    # Select the rows with the click stimulus data
    click_stim_df = raw_df[raw_df[1]=='clickedStim']
    click_stim_df.size

    # Calculate the number of clicks made by the participant
    clicks_observed = click_stim_df.count(axis=1) - 2 
    clicks_observed = clicks_observed.tolist()

    # Calculate the number of clicks that the participant should have made
    clicks_expected =  pd.to_numeric(longest) + 1
    clicks_expected = clicks_expected.tolist()

    # Create a new dataframe with all the values calculated above
    clean_data = pd.DataFrame(data ={'seq length':longest,
                        'errors': error_number,
                        'clicks expected': clicks_expected,
                        'clicks observed':clicks_observed})

    # Return the new dataframe
    return clean_data


### Data Integration For Each Test

In [28]:
def create_merged_df(config_dict):
    data_dict = {}

    # read the files 
    for test, file in config_dict.items():
        df_dict = pd.read_excel(file, sheet_name=None, header=None)

        for session, df in df_dict.items():

            # extracting the participant name and type name
            participant = test.split('_')[-1]
            test_name = test.split('_')[0]

            #extracting repeat number and making its column except for personal
            try:
                type, repeat = session.split('_')
                df.insert(0, 'repeat', repeat)

            except ValueError:
                type = session


            # Running function to clean digit span data
            if test_name == 'digit':
                df = clean_digit_span(df.iloc[3:])
                df.insert(0, 'repeat', repeat)
            
            # verbal fluency test contains header
            elif test_name =='verbal':
                df = df.iloc[1:]

            # inserting the type and participant columns
            df.insert(0, 'type', type)
            df.insert(0, 'participant', participant)
 
            # concatenating data frames of each test
            if test_name not in data_dict:
                data_dict[test_name] = df
            else:
                data_dict[test_name] = pd.concat([data_dict[test_name], df])
    
    return data_dict

data_dict = create_merged_df(config)

In [29]:
# Run the function
df_dict = create_merged_df(config)

---

### Flanker Test Analysis

In [6]:
# creating Flanker dataframe
def create_flanker_dataframe():
    flanker_df = data_dict["flanker"]
    flanker_df.rename(columns={0: "pattern", 1: "expression", 2: "correctness", 3: "response-time"}, inplace=True)
    flanker_df["correctness"] = flanker_df["correctness"].replace(1, "correct")
    flanker_df["correctness"] = flanker_df["correctness"].replace(2, "incorrect")
    flanker_df["correctness"] = flanker_df["correctness"].replace(3, "not-answer")
    return flanker_df

flanker_df = create_flanker_dataframe()

In [7]:
def show_plot(data, title, x_label="", y_label="", palette=["salmon", "skyblue"], factors=["dehydration", "control"]):
    index_cmap = factor_cmap('x', palette=palette, factors=factors, start=1, end=2)
    x = list(data.index.values)
    data_map = {
        'x': x,
        'counts': data.tolist()
        }

    source = ColumnDataSource(data=data_map)
    p = figure(x_range=FactorRange(*x), y_range=(0, 100), height=400, title=title,
               toolbar_location=None, tools="", x_axis_label=x_label, y_axis_label=y_label)

    p.vbar(x='x', top='counts', width=0.9, source=source, fill_color=index_cmap)

    p.y_range.start = 0
    p.x_range.range_padding = 0.1
    p.xaxis.major_label_orientation = 1
    p.xgrid.grid_line_color = None
    return p

def flanker_calculate_counts(flanker_df, answer_type="correct"):
    flanker_df = flanker_df[flanker_df["correctness"] == answer_type]
    flanker_df = flanker_df.groupby(["participant", "type", "repeat"])["correctness"].count().reset_index()
    data = flanker_df.groupby(by=["participant", "type"])["correctness"].mean()
    return data

def flanker_plot_count(answer_type="correct"):  # Roya
    flanker_df = create_flanker_dataframe()
    data = flanker_calculate_counts(flanker_df, answer_type)
    return show_plot(data, f"Average of {answer_type} answers", "participant/session", "count" )

answer_types =['correct','incorrect']
inter_plot = pn.interact(flanker_plot_count, answer_type = answer_types)
inter_plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'897e6739-c154-4714-9514-f00ad9ce80e0': {'defs': …

Column
    [0] Column
        [0] Select(name='answer_type', options=['correct', 'incorrect'], value='correct')
    [1] Row
        [0] Bokeh(Figure, name='interactive00781')

In [8]:
def calculate_percentage(flanker_df, answer_type="correct"):
    df_all = flanker_df.groupby(["participant", "type", "repeat"]).agg(count=("correctness", "count"))
    flanker_df = flanker_df[flanker_df["correctness"] == answer_type]
    df_correct = flanker_df.groupby(["participant", "type", "repeat"]).agg(count=("correctness", "count"))

    flanker_df = round(df_correct["count"] * 100 / df_all["count"], 2).rename("correctness").reset_index().fillna(0)
    data = flanker_df.groupby(by=["participant", "type"])[
        "correctness"].mean()
    return data
    
def flanker_plot_percentage(answer_type="correct"):  # Roya
    flanker_df = create_flanker_dataframe()
    data = calculate_percentage(flanker_df, answer_type)
    return show_plot(data, f"Percentage of {answer_type} answers", "participant/session", "Percentage" )



answer_types =['correct','incorrect']
inter_plot = pn.interact(flanker_plot_percentage, answer_type = answer_types)
inter_plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'a4745fcf-ea3e-4a4f-abc8-58b31d36901d': {'defs': …

Column
    [0] Column
        [0] Select(name='answer_type', options=['correct', 'incorrect'], value='correct')
    [1] Row
        [0] Bokeh(Figure, name='interactive00788')

## Error Plot

In [9]:
def calculate_standard_error(flanker_df, answer_type="correct"):
    flanker_df = flanker_df[flanker_df["correctness"] == answer_type]
    flanker_df = flanker_df.groupby(["participant", "type", "repeat"])["correctness"].count().reset_index()
    df_mean = flanker_df.groupby(by=["participant", "type"]).agg(mean=("correctness", "mean"))
    df_se = flanker_df.groupby(by=["participant", "type"]).agg(se=("correctness", "sem"))
    upper = df_mean["mean"] + 1.96 * df_se["se"]
    lower = df_mean["mean"] - 1.96 * df_se["se"]
    data = pd.concat([upper.rename("upper"), lower.rename("lower")], axis=1)
    return data

def plot_standard_error(plot, data):
    x = list(data.index.values)
    data_map = {
        'x': x,
        'upper': data["upper"].tolist(),
        'lower': data["lower"].tolist()

        }
    source = ColumnDataSource(data=data_map)

    w = Whisker(source=source, base="x", upper="upper", lower="lower", 
            line_color='purple', level="overlay")
    w.upper_head.line_color = 'purple'
    w.lower_head.line_color = 'purple'
    w.upper_head.size = w.lower_head.size = 20
    plot.add_layout(w)
    return plot

def flanker_plot_error(answer_type="correct"):  # Roya
    flanker_df = create_flanker_dataframe()
    data = flanker_calculate_counts(flanker_df, answer_type)
    data_se = calculate_standard_error(flanker_df, answer_type)
    p = show_plot(data, f"Standard Error of {answer_type} answers", "participant/session", "SE" )
    p = plot_standard_error(p, data_se)
    return p


answer_types =['correct','incorrect']
inter_plot = pn.interact(flanker_plot_error, answer_type = answer_types)
inter_plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'33395180-da78-4bdb-afc8-21e2f4b7137d': {'defs': …

Column
    [0] Column
        [0] Select(name='answer_type', options=['correct', 'incorrect'], value='correct')
    [1] Row
        [0] Bokeh(Figure, name='interactive00795')

### Stroop Test  Analysis

In [10]:
def stroop_test(): # Mahdiye
    total_dict = create_merged_df(config)
    stroop_df = total_dict['stroop']
    stroop_df.drop(stroop_df.columns[[3,7]], axis=1, inplace=True)
    stroop_df = stroop_df.set_axis(['participant', 'type','repeat','word name','word color',
                                    'name_color match','pressed _key','status','reaction_time'], axis=1)
    stroop_df['type&repeat'] = stroop_df['type']+stroop_df['repeat']
    return stroop_df

stroop_df = stroop_test()

In [11]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Whisker
import panel as pn
from bokeh.io import output_notebook

output_notebook()
pn.extension()

Loading BokehJS ...

In [12]:
def individual_stroop_bar_plot(participant='blue'):
    df = stroop_df[stroop_df['participant']==participant]
    
    dff= df.groupby('type&repeat').min().reset_index()
    p = figure(x_range=dff['type&repeat'], height=350, toolbar_location=None, 
               title=f'Stroop Test {participant}', y_axis_label="Reaction time(milliseconds)")
    p.vbar(x=dff['type&repeat'], bottom=0,top=dff['reaction_time'], width=0.5, line_color='white', color=participant)
    return p

#interactive plots
participants_color =['blue','red','orange','green','pink']
inter_plot = pn.interact(individual_stroop_bar_plot, participant = participants_color)
inter_plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'1fccb371-6b70-421d-b246-bb8a3af28fa3': {'defs': …

Column
    [0] Column
        [0] Select(name='participant', options=['blue', 'red', ...], value='blue')
    [1] Row
        [0] Bokeh(Figure, name='interactive00803')

In [13]:
def individual_stroop_box_plot(participant):
    
    df = stroop_df[stroop_df['participant']==participant]
    kinds = df['type&repeat'].unique()
    
    # compute quantiles
    qs = df.groupby('type&repeat').reaction_time.quantile([0.25, 0.5, 0.75])
    qs = qs.unstack().reset_index()
    qs.columns = ['type&repeat', "q1", "q2", "q3"]
    df = pd.merge(df, qs, on='type&repeat', how="left")

    # compute IQR outlier bounds
    iqr = df.q3 - df.q1
    df["upper"] = df.q3 + 1.5*iqr
    df["lower"] = df.q1 - 1.5*iqr

    source = ColumnDataSource(df)

    p = figure(x_range=kinds,y_range=[0,stroop_df['reaction_time'].max() * 1.3],tools="", toolbar_location=None,
                title="box plot of stroop test "+participant,
               background_fill_color="#eaefef", y_axis_label="Reaction time(milliseconds)")


    # outlier range
    whisker = Whisker(base='type&repeat', upper="upper", lower="lower", source=source)
    whisker.upper_head.size = whisker.lower_head.size = 20
    p.add_layout(whisker)

    # quantile boxes
    p.vbar('type&repeat', 0.5, "q2", "q3", color = participant,bottom=0, source=source, line_color="black")
    p.vbar('type&repeat', 0.5, "q1", "q2", color=participant, bottom=0, source=source, line_color="black")
    
    # outliers
    outliers = df[~df.reaction_time.between(df.lower, df.upper)]
    p.scatter('type&repeat', 'reaction_time', source=outliers, size=6, color="black", alpha=0.5)

    p.xgrid.grid_line_color = None
    p.axis.major_label_text_font_size="14px"
    p.axis.axis_label_text_font_size="12px"

    return p
    
#interactive plots
participants_color =['blue','red','orange','green','pink']
inter_plot = pn.interact(individual_stroop_box_plot, participant = participants_color)
inter_plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'6c11afd1-aee8-4b2b-b268-2c00051cbd7f': {'defs': …

Column
    [0] Column
        [0] Select(name='participant', options=['blue', 'red', ...], value='blue')
    [1] Row
        [0] Bokeh(Figure, name='interactive00810')

In [14]:
def total_bar_stroop():

    df = stroop_df
    # create a list of participants
    participants = list(df['participant'].unique())
    
    #create a list of different session types
    types = list(df['type'].unique())
    
    dff = df.groupby(['participant','type']).mean().reset_index()
    
    #create two list of reaction time regarding session types
    control_mean = list(dff[dff['type'] =='control'].reaction_time)
    dehydration_mean = list(dff[dff['type'] =='dehydration'].reaction_time)

    #create a dictionary of 3 keys and values and then convert into a dataframe
    data = {'participants' : participants,
            'control'   : control_mean,
            'dehydration'   : dehydration_mean,
            }
    data = pd.DataFrame(data)

    palette =  ["#718dbf", "#e84d60"] #colors

    # create a list like:
    # [ ("blue", "control"), ("Ablue", "dehydration"), ("red", "control"), ("red", "dehydration"), ... ]
    x = [ (participant, test) for participant in participants for test in types ]
    counts = sum(zip(data['control'], data['dehydration']), ()) # like an hstack

    source = ColumnDataSource(data=dict(x=x, counts=counts))
    # plot
    p = figure(x_range=FactorRange(*x), width=900, height=500, title='Average of reaction time stroop test',
               toolbar_location=None,y_axis_label="Reaction time(milliseconds)", x_axis_label="participant, session type")

    p.vbar(x='x', top='counts', width=1, source=source, line_color="white",
           fill_color=factor_cmap('x', palette=palette, factors=types, start=1, end=2))

    p.y_range.start = 0
    p.x_range.range_padding = 0.1
    p.xaxis.major_label_orientation = 1
    p.xgrid.grid_line_color = None

    return(p)

show(total_bar_stroop())

In [15]:
def error_bar(df = stroop_df, required = 'reaction_time',test_name = 'stroop_test'):
    
    df['participant_type'] = df['participant']+' '+df['type']

    dff = df.groupby('participant_type').mean().reset_index()

    std = df.groupby('participant_type').std().reset_index()
    n = df.groupby('participant_type').count().reset_index()

    name_list = list(dff['participant_type'])
    mean_list = list(dff[required])
    std_list = list(std[required])
    n_list = list(n[required])
    se_list = std_list/np.sqrt(n_list)

    upper = mean_list + se_list*1.96
    lower = mean_list - se_list*1.96

    data = {'participants_type' : name_list,
            'required_mean' : mean_list,
            'SE' : se_list,
            'upper' : upper,
            'lower' : lower,
            }
    data = pd.DataFrame(data)

    source = ColumnDataSource(data)

    # plot

    p = figure(x_range= name_list, y_range = [data['lower'].min()-100,data['upper'].max()+100],  width=900, 
               height=500, title="Error bar "+ test_name, toolbar_location=None, tools="")

    p.circle(x='participants_type', y= 'required_mean', width=0.9, color='blue', source=source)

    whisker = Whisker(base='participants_type', upper="upper", lower="lower", source=source)
    whisker.upper_head.size = whisker.lower_head.size = 20
    p.add_layout(whisker)
    return p

show(error_bar(df = stroop_df, required = 'reaction_time',test_name = 'stroop_test'))


### Stop Signal Analysis

In [16]:

column_meanings = {'Column':[0,1,2,3,4,5,6,7],
                   'Meaning':['trial type (go or nogo)', 
                              'required response (left or right)', 
                              'when the stop signal is shown (or 0 if not)', 
                              'response time 1', 
                              'status 1 (1=correct, 2=wrong, 3=timeout)',
                              'response time 2 (only in no go trials)',
                              'status 2 (only in no go trials; 1=correct, 2=wrong, 3=timeout)',
                              '1=trial is correct ; 0=trial is not correct']} 

column_meanings = pd.DataFrame(column_meanings)
column_meanings.set_index('Column', inplace=True)
column_meanings


,Meaning
Column,
0,trial type (go or nogo)
1,required response (left or right)
2,when the stop signal is shown (or 0 if not)
3,response time 1
4,"status 1 (1=correct, 2=wrong, 3=timeout)"
5,response time 2 (only in no go trials)
6,"status 2 (only in no go trials; 1=correct, 2=w..."
7,1=trial is correct ; 0=trial is not correct


In [17]:
def stop_test(stop_df): # Jacob
    
    # renaming and reordering columns
    stop_df.rename(columns = {0:'trial_type', 1:'correct_resp.', 
                            2:'stop_signal_delay', 3:'response_time',
                            4:'status', 5:'resonse_time_nogo',
                            6:'status_nogo', 7:'correct'}, inplace = True)

    stop_df = stop_df[['participant', 'type', 'repeat', 'trial_type',
                    'correct_resp.', 'correct', 'response_time',
                    'status', 'stop_signal_delay', 'resonse_time_nogo',
                    'status_nogo']]

    # The average resonse time for go trials per trial type
    avg_go_resp_time = stop_df[stop_df['trial_type'] == 'go'].groupby([
        'participant', 'type','status']).mean()['response_time']


    # The average resonse time for no-go trials per correct/incorrect trial
    avg_nogo_resp_time = stop_df[stop_df['trial_type'] == 'nogo'].groupby([
        'participant', 'type','status_nogo']).mean()['response_time']

    # Good to keep in mind that here, status three corresponds with a correct trail
    # Since there was no press in a no-go trial.

    # Number of errors and time-outs in go trials
    errors_timeout_go = stop_df[(stop_df['trial_type'] == 'go') & 
                                (stop_df['status'] != 1.0)].groupby([
                                    'participant', 'type', 'repeat','status']).count()['trial_type']

    # Number of errors and time-outs in no-go trials
    errors_timeout_nogo = stop_df[stop_df['trial_type'] == 'nogo'].groupby([
        'participant', 'type', 'repeat','status_nogo']).count()['trial_type']
    
    return avg_go_resp_time, avg_nogo_resp_time, errors_timeout_go, errors_timeout_nogo

# callig the function
avg_go_resp_time, avg_nogo_resp_time, errors_timeout_go, errors_timeout_nogo = stop_test(data_dict['stop'])


In [18]:
react_go_boxplot = data_dict['stop'][(data_dict['stop']['trial_type'] == 'go') & 
                                     (data_dict['stop']['correct'] == 1)][['response_time', 'participant', 'type']
                                                      ].hvplot.box(by='type', 
                                                                   groupby='participant',
                                                                   title='Reaction time for correct responses',
                                                                   xlabel='Session Type', 
                                                                   ylabel='Resopnse Time (ms)')

react_go_boxplot

BokehModel(combine_events=True, render_bundle={'docs_json': {'a7081f0a-ca3e-4b03-9a13-deb16ccc5697': {'defs': …

:DynamicMap   [participant]
   :BoxWhisker   [type]   (response_time)

In [19]:
# TODO: calculate percentage of errors/correct

participants = ['blue', 'green', 'red', 'pink', 'orange']
session_type = ['control', 'dehydratation']

perc_correct = pd.DataFrame(index=[participants])


In [20]:
df = data_dict['stop']
correct = len(df[(df['participant'] == 'blue') &
    (df['type'] == 'dehydration') &
    (df['correct'] == 1)])
total = len(df[(df['participant'] == 'blue') &
    (df['type'] == 'dehydration')])

perc_correct = (correct/total) * 100
perc_correct

86.42857142857143

### Verbal Fluency Analysis

In [21]:
verbal_df = data_dict['verbal'].copy()
verbal_df = verbal_df[verbal_df[1] != 'word count'] # to remove silly headers
verbal_df.rename(columns={0:'word_type', 1:'n'}, inplace=True)
verbal_df['n'] = verbal_df['n'].astype(int)

verbal_avg = verbal_df.groupby(['participant', 'type']).mean().round(2)
error_data = verbal_df.describe().transpose()

verbal_avg_bar = verbal_avg.hvplot.bar(title='Average number of words produced per session type',
                                        xlabel='Participant, Session Type', 
                                        ylabel ='Number of words').opts(xrotation=25)# * error_data.hvplot.errorbars(y='max', yerr1='std')

In [22]:
def verbal_test(verbal_df): # Jacob
    verbal_df = data_dict['verbal'].copy()
    verbal_df = verbal_df[verbal_df[1] != 'word count'] # to remove silly headers
    verbal_df.rename(columns={0:'word_type', 1:'n'}, inplace=True)
    verbal_df['n'] = verbal_df['n'].astype(int)

    verbal_avg = verbal_df.groupby(['participant', 'type']).mean().round(2)
    
    verbal_avg_bar = verbal_avg.hvplot.bar(title='Average number of words produced per session type',
                                           xlabel='Participant, Session Type', 
                                           ylabel ='Number of words').opts(xrotation=25)# * verbal_avg.hvplot.errorbars(x=)

    return verbal_df, verbal_avg, verbal_avg_bar

verbal_df, verbal_avg, verbal_avg_bar = verbal_test(data_dict['verbal'])
verbal_avg_bar

:Bars   [participant,type]   (n)

### Digit Span Analysis

In [32]:
def digit_test(digit_df): # Karina
    '''Function to analyse the Digit Span data and to create a dataframe with the information needed to plot'''
    # Change data types
    digit_df = digit_df.astype({'participant': 'string',
                                'type': 'string',
                                'repeat': 'int',
                                'seq length':'float',
                                'errors': 'float',
                                'clicks expected': 'float',
                                'clicks observed': 'float'})

    # Make calculations by column taking the groups into account               
    digit_grouped = digit_df.groupby(['participant','type', 'repeat']).agg({'seq length': 'max','errors': 'sum', 'clicks expected':'max','clicks observed':'max'} )

    # Calculte values needed for the analysis
    digit_grouped['clicks difference'] = digit_grouped['clicks observed'] - digit_grouped['clicks expected']
    digit_grouped['seq length'] = digit_grouped['seq length'] - 1

    # Obtain mean and estandar error
    digit_mean_sem = digit_grouped.groupby(['participant', 'type']).agg(['mean','sem'])
    digit_mean_sem

    return digit_mean_sem

#Running function
digit_mean_sem = digit_test(digit_df = df_dict["digit"])
print(digit_mean_sem)

def digit_barplots(participant, analysis):
    '''Function to plot the digit span data'''

    df = digit_mean_sem#digit_mean_sem.loc[participant]
    participants = df.rese
    sessions_type = df.reset_index().type.unique().tolist()
    values = df[analysis]['mean'].tolist()
    palette=[participant, 'grey']
    y_label = {'seq length': 'Longest sequence remembered (mean)',
              'errors':'Number of errors made (mean)'}

    x = [(participant, session) for session in sessions_type]
    print(x)
    source = ColumnDataSource(data=dict(x=x, counts=values))

    p = figure(x_range=FactorRange(*x), height=550, title="Digit Span",
            toolbar_location=None, tools="")

    p.vbar(x='x', top='counts', width=0.9, source=source, line_color="white",
        fill_color=factor_cmap('x', palette=palette, factors=sessions_type, start=1, end=2))

    # x-axis
    p.xaxis.axis_label = "Participant"
    p.x_range.range_padding = 0.1
    p.xaxis.major_label_orientation = 0
    p.xgrid.grid_line_color = None

    #  y-axis
    p.yaxis.axis_label = y_label[analysis]
    p.y_range.start = 0
    p.yaxis.major_label_orientation = "vertical"
    p.y_range.range_padding = 0.3
    
    return (p)

p1 = digit_barplots('blue', 'seq length')
p1
# p2 = digit_barplots('green', 'seq length')
# p3 = digit_barplots('orange', 'seq length')
# p4 = digit_barplots('pink', 'seq length')
# p5 = digit_barplots('red', 'seq length')

# p6 = digit_barplots('blue', 'errors')
# p7 = digit_barplots('green', 'errors')
# p8 = digit_barplots('orange', 'errors')
# p9 = digit_barplots('pink', 'errors')
# p10 = digit_barplots('red', 'errors')

#show(gridplot([p1,p2, p3, p4, p5], ncols=2, width=500, height=500))
#show(gridplot([p6,p7, p8, p9, p10], ncols=2, width=500, height=500))

# # Making interactive the plots 
# participants_colour = ['blue', 'green', 'orange', 'pink', 'red']
# analyses = ['seq length', 'errors']
# inter_plot = pn.interact(digit_barplots, participant=participants_colour, analysis=analyses)
# inter_plot

# # Creating a dashboard
# dashboard = pn.template.BootstrapTemplate(title='Title', sidebar_with = 400)
# dashboard.sidebar.append(inter_plot[0])
# dashboard.main.append(inter_plot[1])
# dashboard.show()


                        seq length      errors      clicks expected       \
                              mean  sem   mean  sem            mean  sem   
participant type                                                           
blue        control            6.0  0.0    2.0  0.0             8.0  0.0   
            dehydration        6.0  1.0    3.0  0.0             8.0  1.0   
green       control            6.0  0.0    2.5  0.5             8.0  0.0   
            dehydration        5.5  0.5    2.0  0.0             7.5  0.5   
orange      control            5.5  0.5    2.0  0.0             7.5  0.5   
            dehydration        5.5  0.5    2.5  0.5             7.5  0.5   
pink        control            6.5  0.5    3.0  1.0             8.5  0.5   
            dehydration        5.5  0.5    2.5  0.5             7.5  0.5   
red         control            7.5  0.5    3.0  0.0             9.5  0.5   
            dehydration        6.5  0.5    2.0  0.0             8.5  0.5   

           

Figure(id='4165', ...)

### Dehydration percentage

In [24]:
# Obtain personal data
df_health = df_dict['personal']
# Theres one missing value in the green session. Fill it with the correct value
df_health[0] = df_health[0].fillna(2)

def dehydration_percentage():
    '''Function to obtain the % of dehydration per participant per session'''
    # Keep only the body weight data and drop NaN
    body_weight = df_health[['participant','type', 0, 5]].dropna()
    # drop the rows with strings on them (the ones with index ==0 )
    body_weight = body_weight[body_weight.index!=0]
    # Rename the columns
    body_weight.rename(columns={0:'session', 5:'body weight'}, inplace=True)

    body_weight = body_weight.astype({'body weight':'float'})
    # calculate percentage of dehydration
    body_weight['dehydration %'] = (body_weight.groupby(['participant','type','session'])['body weight'].pct_change()) * 100
    return body_weight

dehydration_percentage()

,participant,type,session,body weight,dehydration %
1,blue,control,1,73.7,NaN
9,blue,control,1,74.0,0.407056
10,blue,control,2,72.3,NaN
18,blue,control,2,72.2,-0.138313
1,blue,dehydration,1,72.6,NaN
9,blue,dehydration,1,72.8,0.275482
10,blue,dehydration,2,73.3,NaN
18,blue,dehydration,2,72.8,-0.682128
1,green,control,1,61.5,NaN
9,green,control,1,61.3,-0.325203


---